In [1]:
import importlib
import sys

sys.path.append('/Users/hildahuo/Desktop/course registraition/research project/VAR/var python code/Gibbs_sampler/Gibbs_with_numba')

import SMC_square_parallelized as SMC_square_parallelized # assuming this is already imported
importlib.reload(SMC_square_parallelized)



<module 'SMC_square_parallelized' from '/Users/hildahuo/Desktop/course registraition/research project/VAR/var python code/Gibbs_sampler/Gibbs_with_numba/SMC_square_parallelized.py'>

In [2]:
sys.path.append('/Users/hildahuo/Desktop/course registraition/research project/VAR/var python code/Gibbs_sampler')
import prior_setup
importlib.reload(prior_setup)
from prior_setup import BayesianVARPrior

In [2]:
import importlib
import sys
import SMC_square_parallelized
importlib.reload(SMC_square_parallelized)

from SMC_square_parallelized import WF_SMC_Square_with_tree

In [28]:
import pandas as pd
df = pd.read_csv("/Users/hildahuo/Desktop/course registraition/research project/VAR/var python code/other code/synthesized_data.csv")
y = df.values
print(y.shape)

(600, 5)


In [3]:
import sys
import importlib
sys.path.append('/Users/hildahuo/Desktop/course registraition/research project/VAR/var python code/experiment')
import fetch_fred_data_bygroup
importlib.reload(fetch_fred_data_bygroup)
from fetch_fred_data_bygroup import fetch_fred_data

data = fetch_fred_data ()

/Users/hildahuo/Desktop/course registraition/research project/VAR/var python code/experiment/fetch_fred_data_bygroup.py:53: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  expected_length = pd.date_range(start=start_date, end=end_date, freq='M').shape[0]


(495,)
(495,)
(495,)
(495,)
(495,)
(495,)
(495,)
(495,)
(495,)
(495,)
(495,)
(495,)
(495,)
(495,)
(495,)
(495,)
Final dataframe shape: (495, 16)
Length time series: (495, 16)
Number of data points NOT used for prediction (training): 240
Number of data points used FOR prediction:                255


/Users/hildahuo/Desktop/course registraition/research project/VAR/var python code/experiment/fetch_fred_data_bygroup.py:116: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_cleaned = data_cleaned.fillna(method='bfill').fillna(method='ffill')


In [4]:
y = data.values

In [5]:
import psutil
import time
import logging

def monitor_resources(log_interval=5, stop_event=None, tag="SMC2"):
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
    process = psutil.Process()
    while stop_event is None or not stop_event.is_set():
        cpu = psutil.cpu_percent()
        mem = psutil.virtual_memory()
        rss = process.memory_info().rss / 1e9  # in GB
        logging.info(f"[{tag}] CPU: {cpu:.1f}% | RAM: {mem.percent:.1f}% used ({rss:.2f} GB RSS)")
        time.sleep(log_interval)

import threading

# Create a stop event
stop_event = threading.Event()

# Start resource monitor in background
monitor_thread = threading.Thread(
    target=monitor_resources, 
    kwargs={"log_interval": 5, "stop_event": stop_event, "tag": "SMC2"},
    daemon=True
)
monitor_thread.start()

2025-12-04 17:38:20,755 - [SMC2] CPU: 23.0% | RAM: 79.6% used (0.19 GB RSS)


In [50]:
y = data.values

In [6]:
import pickle
import os
try:
    output_dir = "smc2_run_outputs"
    os.makedirs(output_dir, exist_ok=True)

    all_results = []  # Collect results for all runs here
    base_seed = 0
    num_runs = 1
    N_x = 30

    for run_i in range(num_runs):
        smc2_instance = WF_SMC_Square_with_tree(
        p= 13,
        logfile=f"log_run_{run_i}_N_x_{N_x}_K3_p4_AR1.txt",
        N_theta= 50, # keep M_theta = 15 for the memory
        N_x=N_x,
        N_rejuvenate= 9,
        update_A = True, # test with A
        )
        
        # increase N_x
        smc2_instance.run(y=y, t_init= 100, if_pred = False, step_ahead = 1) #, init_result = init_syn_data)

        run_result = {
        "run_id": run_i,
        "N_x": N_x,
        "N_theta": 7500,
        #"likelihood_track": smc2_instance.likelihood_track,
        #"predicted_residuals": smc2_instance.predicted_residuals,
        "ESS_track": smc2_instance.ESS_track,
        'Residuals' : smc2_instance.predictions,
        #'Pi_stability': smc2_instance.pi_eigen,
        #'A': smc2_instance.A,
        #'Pi': smc2_instance.Pi,
        #'Phi': smc2_instance.Phi,
        #'mu' : smc2_instance.mu,
        #'rho': smc2_instance.rho,
        #'trajactories': smc2_instance.trajectories,
        #'ancestors': smc2_instance.ancestors,
        #'outer weights': smc2_instance.outer_weights,
        #'inner weights': smc2_instance.inner_weights,
        'mem_usage' : smc2_instance.mem_usage,
        }

        all_results.append(run_result)
        print(f"[✓] Run {run_i} finished.")

    # Save everything in one file
    file_path = os.path.join(output_dir, f"all_runs_N_x_{N_x}_K3_p4_mem_test.pkl")
    with open(file_path, "wb") as f:
        pickle.dump(all_results, f)

    print(f"[✓] All {num_runs} runs saved to {file_path}")

finally:
    # Stop the resource monitor gracefully
    stop_event.set()
    monitor_thread.join()
    print("Resource monitoring stopped.")

482


2025-12-04 17:38:25,762 - [SMC2] CPU: 97.8% | RAM: 82.5% used (0.09 GB RSS)
2025-12-04 17:38:30,767 - [SMC2] CPU: 74.2% | RAM: 80.2% used (0.20 GB RSS)
2025-12-04 17:38:35,772 - [SMC2] CPU: 75.6% | RAM: 81.0% used (0.19 GB RSS)
2025-12-04 17:38:40,777 - [SMC2] CPU: 71.3% | RAM: 81.0% used (0.18 GB RSS)
2025-12-04 17:38:45,782 - [SMC2] CPU: 70.4% | RAM: 80.8% used (0.16 GB RSS)
2025-12-04 17:38:50,786 - [SMC2] CPU: 69.0% | RAM: 80.9% used (0.16 GB RSS)
2025-12-04 17:38:55,793 - [SMC2] CPU: 71.4% | RAM: 80.8% used (0.16 GB RSS)
2025-12-04 17:39:00,800 - [SMC2] CPU: 72.9% | RAM: 82.8% used (0.12 GB RSS)


 24%|██▍       | 122/510 [00:38<02:09,  2.99it/s]

Resource monitoring stopped.


KeyboardInterrupt: 